In [21]:
import os
import numpy as np
import librosa
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Dropout
import tensorflow.keras
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

In [2]:
DATASET_PATH = "/Users/zihuiouyang/Downloads/LA/ASVspoof2019_LA_train/flac"
LABEL_FILE_PATH = "/Users/zihuiouyang/Downloads/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.train.trn.txt"
NUM_CLASSES = 2  # Number of classes (bonafide and spoof)
SAMPLE_RATE = 16000  # Sample rate of your audio files
DURATION = 5  # Duration of audio clips in seconds
N_MELS = 128  # Number of Mel frequency bins

In [3]:
import fnmatch
import os

In [4]:
a = []
for filename in os.listdir('/Users/zihuiouyang/Downloads/cv-corpus-15.0-delta-2023-09-08/en/clips'):
    if fnmatch.fnmatch(filename, '*.flac'):
        a.append(filename)

In [5]:
import random
a1 = random.sample(a,22800)

In [6]:
labels = {}
for i in range(len(a1)):
    file_name = a1[i]
    label = 1
    labels[file_name] = label

In [36]:
X = []
y = []

max_time_steps = 250  # Define the maximum time steps for your model

for file_name, label in labels.items():
    file_path = os.path.join("/Users/zihuiouyang/Downloads/cv-corpus-15.0-delta-2023-09-08/en/clips", file_name)

    # Load audio file using librosa
    audio, _ = librosa.load(file_path, sr=SAMPLE_RATE, duration=DURATION)

    # Extract Mel spectrogram using librosa
    mel_spectrogram = librosa.feature.melspectrogram(y=audio, sr=SAMPLE_RATE, n_mels=N_MELS)
    mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)

    # Ensure all spectrograms have the same width (time steps)
    if mel_spectrogram.shape[1] < max_time_steps:
        mel_spectrogram = np.pad(mel_spectrogram, ((0, 0), (0, max_time_steps - mel_spectrogram.shape[1])), mode='constant')
    else:
        mel_spectrogram = mel_spectrogram[:, :max_time_steps]

    X.append(mel_spectrogram)
    y.append(label)

In [37]:
labels1 = {}
with open(LABEL_FILE_PATH, 'r') as label_file:
    lines = label_file.readlines()

for line in lines:
    parts = line.strip().split()
    file_name = parts[1]
    if parts[-1] == "bonafide":
        continue
    label = 0
    labels1[file_name] = label

In [38]:
max_time_steps = 250  # Define the maximum time steps for your model

for file_name, label in labels1.items():
    file_path = os.path.join(DATASET_PATH, file_name + ".flac")

    # Load audio file using librosa
    audio, _ = librosa.load(file_path, sr=SAMPLE_RATE, duration=DURATION)

    # Extract Mel spectrogram using librosa
    mel_spectrogram = librosa.feature.melspectrogram(y=audio, sr=SAMPLE_RATE, n_mels=N_MELS)
    mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)

    # Ensure all spectrograms have the same width (time steps)
    if mel_spectrogram.shape[1] < max_time_steps:
        mel_spectrogram = np.pad(mel_spectrogram, ((0, 0), (0, max_time_steps - mel_spectrogram.shape[1])), mode='constant')
    else:
        mel_spectrogram = mel_spectrogram[:, :max_time_steps]

    X.append(mel_spectrogram)
    y.append(label)

In [39]:
X = np.array(X)
y = np.array(y)
y_encoded = to_categorical(y, NUM_CLASSES)
split_index = int(0.8 * len(X))
b = []
for i in range(45600):
    b.append(i)
b1 = random.sample(b,36480)
mask=np.full(len(b),False,dtype=bool)
mask[b1]=True
X_train, X_val = X[mask], X[~mask]
y_train, y_val = y_encoded[mask], y_encoded[~mask]

In [40]:
input_shape = (N_MELS, X_train.shape[2])

In [41]:
model = tf.keras.Sequential()
model.add(LSTM(128,input_shape=input_shape))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(48, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2, activation='softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 128)               194048    
                                                                 
 dropout_6 (Dropout)         (None, 128)               0         
                                                                 
 dense_8 (Dense)             (None, 128)               16512     
                                                                 
 dense_9 (Dense)             (None, 64)                8256      
                                                                 
 dropout_7 (Dropout)         (None, 64)                0         
                                                                 
 dense_10 (Dense)            (None, 48)                3120      
                                                                 
 dropout_8 (Dropout)         (None, 48)               

In [42]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [43]:
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10


2023-10-31 10:03:33.706971: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1140/1140 [==============================] - 89s 72ms/step - loss: 0.6382 - accuracy: 0.5639 - val_loss: 6.1694 - val_accuracy: 0.4902
Epoch 2/10
1140/1140 [==============================] - 80s 70ms/step - loss: 0.6373 - accuracy: 0.5664 - val_loss: 6.3593 - val_accuracy: 0.4902
Epoch 3/10
1140/1140 [==============================] - 82s 72ms/step - loss: 0.7044 - accuracy: 0.5550 - val_loss: 10.2207 - val_accuracy: 0.4902
Epoch 4/10
1140/1140 [==============================] - 81s 71ms/step - loss: 0.6592 - accuracy: 0.5490 - val_loss: 9.8645 - val_accuracy: 0.4902
Epoch 5/10
1140/1140 [==============================] - 80s 70ms/step - loss: 0.6535 - accuracy: 0.5542 - val_loss: 10.1938 - val_accuracy: 0.4902
Epoch 6/10
1140/1140 [==============================] - 82s 72ms/step - loss: 0.6524 - accuracy: 0.5543 - val_loss: 9.7608 - val_accuracy: 0.4902
Epoch 7/10
1140/1140 [==============================] - 82s 72ms/step - loss: 0.6530 - accuracy: 0.5508 - val_loss: 9.8448 - val_accu

In [44]:
model.save("/Users/zihuiouyang/Documents/audio_classifier.h5")

/Users/zihuiouyang/anaconda3/envs/coqui/lib/python3.9/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [45]:
TEST_DATASET_PATH = "./TestEvaluation"
MODEL_PATH = "/Users/zihuiouyang/Documents/audio_classifier.h5"  # Replace with the actual path to your saved model
SAMPLE_RATE = 16000
DURATION = 5
N_MELS = 128
MAX_TIME_STEPS = 250

In [46]:
from tensorflow.keras.models import load_model
model = load_model(MODEL_PATH)

In [47]:
X_test = []

test_files = os.listdir(TEST_DATASET_PATH)
for file_name in test_files:
    file_path = os.path.join(TEST_DATASET_PATH, file_name)

    # Load audio file using librosa
    audio, _ = librosa.load(file_path, sr=SAMPLE_RATE, duration=DURATION)

    # Extract Mel spectrogram using librosa
    mel_spectrogram = librosa.feature.melspectrogram(y=audio, sr=SAMPLE_RATE, n_mels=N_MELS)
    mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)

    # Ensure all spectrograms have the same width (time steps)
    if mel_spectrogram.shape[1] < MAX_TIME_STEPS:
        mel_spectrogram = np.pad(mel_spectrogram, ((0, 0), (0, MAX_TIME_STEPS - mel_spectrogram.shape[1])), mode='constant')
    else:
        mel_spectrogram = mel_spectrogram[:, :MAX_TIME_STEPS]

    X_test.append(mel_spectrogram)

# Convert list to numpy array
X_test = np.array(X_test)

# Predict using the loaded model
y_pred = model.predict(X_test)

# Convert probabilities to predicted classes
y_pred_classes = np.argmax(y_pred, axis=1)

y_pred

1/1 [==============================] - 1s 1s/step


array([[1.9054447e-09, 1.0000000e+00],
       [7.7551896e-08, 9.9999988e-01],
       [2.4914377e-09, 1.0000000e+00],
       [2.2560271e-09, 1.0000000e+00],
       [1.9114728e-09, 1.0000000e+00],
       [2.4700697e-09, 1.0000000e+00],
       [1.9063242e-09, 1.0000000e+00],
       [3.8757135e-09, 1.0000000e+00],
       [2.5543032e-09, 1.0000000e+00],
       [2.0504369e-09, 1.0000000e+00]], dtype=float32)